Import libraries for data cleaning 

In [21]:
import csv
from tokenize import String

from db_connection import get_db_engine
from db_connection import execute_query
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

Reading CSV file

In [22]:
df_releases = pd.read_csv(r"C:\Users\ricca\Desktop\CSV\releases.csv\releases.csv")

Top records of CSV file: releases.csv

In [23]:
df_releases

,id,country,date,type,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN
...,...,...,...,...,...
1332777,1940967,USA,1909-01-01,Theatrical,NaN
1332778,1940968,Sweden,1908-11-11,Theatrical,NaN
1332779,1940969,France,1902-01-01,Theatrical,NaN
1332780,1940970,France,1902-01-01,Theatrical,NaN


Checking for total dimension of the csv

In [24]:
df_releases.shape

(1332782, 5)

Rename 'id' column in 'id_actor'. "inplace = True" modifies directly the df without creating a copy

In [25]:
df_releases.rename(columns = {"id":"id_release"} , inplace = True)


In [26]:
df_releases.columns

Index(['id_release', 'country', 'date', 'type', 'rating'], dtype='object')

Check info about column type

In [27]:
df_releases.dtypes

id_release     int64
country       object
date          object
type          object
rating        object
dtype: object

Correction of data type of the records

In [28]:
df_releases['id_release'] = df_releases['id_release'].astype('Int64')
df_releases['country'] = df_releases['country'].fillna(pd.NA).astype('string')
df_releases["date"] = pd.to_datetime(df_releases["date"], errors="coerce")
df_releases['type'] = df_releases['type'].fillna(pd.NA).astype('string')
df_releases['rating'] = df_releases['rating'].fillna(pd.NA).astype('string')
df_releases.dtypes

id_release             Int64
country       string[python]
date          datetime64[ns]
type          string[python]
rating        string[python]
dtype: object

Check if there are duplicates

In [42]:
df_releases.duplicated(subset=["id_release", "country", "date", "type"]).sum()


np.int64(5421)

In [43]:
df_releases.drop_duplicates(subset=["id_release", "country", "date", "type"], inplace=True)
df_releases.duplicated().sum()

np.int64(0)

Check the missing values

In [44]:
df_releases.isnull().sum()

id_release         0
country            0
date               0
type               0
rating        998802
dtype: int64

Dropping records with missing value 

In [45]:
df_releases.dropna(subset=['id_release', 'country', 'date', 'type'], inplace = True)
df_releases.isnull().sum()

id_release         0
country            0
date               0
type               0
rating        998802
dtype: int64

Connecting to PostgreSQL database and creating the table

In [46]:
engine = get_db_engine()
sql = \
    """CREATE TABLE IF NOT EXISTS releases (
            id_release INTEGER CHECK (id_release>=0),
            country TEXT CHECK (country<>''),
            date DATE,
            type TEXT CHECK (type<>''),
            rating TEXT,
            

            PRIMARY KEY (id_release, country, date, type)
            );
    """
execute_query(sql)

Success!


Population of the table using .to_sql function

In [47]:
df_releases.to_sql("releases", engine, if_exists="append", index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) ERRORE:  un valore chiave duplicato viola il vincolo univoco "releases_pkey"
DETAIL:  La chiave (id_release, country, date)=(1000002, Ireland, 2020-07-24) esiste già.

[SQL: INSERT INTO releases (id_release, country, date, type, rating) VALUES (%(id_release__0)s, %(country__0)s, %(date__0)s, %(type__0)s, %(rating__0)s), (%(id_release__1)s, %(country__1)s, %(date__1)s, %(type__1)s, %(rating__1)s), (%(id_release__2)s, %(co ... 87168 characters truncated ... ting__998)s), (%(id_release__999)s, %(country__999)s, %(date__999)s, %(type__999)s, %(rating__999)s)]
[parameters: {'rating__0': None, 'type__0': 'Theatrical', 'country__0': 'Andorra', 'date__0': datetime.datetime(2023, 7, 21, 0, 0), 'id_release__0': 1000001, 'rating__1': 'ATP', 'type__1': 'Theatrical', 'country__1': 'Argentina', 'date__1': datetime.datetime(2023, 7, 20, 0, 0), 'id_release__1': 1000001, 'rating__2': 'PG', 'type__2': 'Theatrical', 'country__2': 'Australia', 'date__2': d